In [1]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import os
import tensorflow as tf
from config import root_dir, model_to_run, bottlenecks, concepts, version, num_random_exp, max_examples, run_parallel, num_workers, is_cav_on, make_random,true_cav
import sys
import numpy as np
import importlib
importlib.reload(act_gen)
from tcav.utils_analysis import get_cav_mean, get_true_cav_mean, get_sensitivity, get_score, get_predict, get_grad, get_logit_grad, cos_sim, get_cav
from PIL import Image

In [2]:
# target
save_folder = model_to_run
working_dir = root_dir + 'log/'  + save_folder
activation_dir =  working_dir+ '/activations/'
source_dir = root_dir + 'tcav/dataset/for_tcav/'
cav_dir = working_dir + '/cavs/'
# this is a regularizer penalty parameter for linear classifier to get CAVs.
alphas = [0.1]

sess = utils.create_session()

#===============================================================================
GRAPH_PATH = root_dir + 'tcav/frozen_models/colored_mnist_number_2layers_cnn.pb'
#LABEL_PATH = root_dir + 'tcav/dataset/colored_mnist-color-number'
LABEL_PATH = root_dir + 'tcav/dataset/colored_mnist_number'

mymodel = model.KerasMnistCnnWrapper_public(sess,GRAPH_PATH,LABEL_PATH)
#=================================================================================

act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=max_examples)






You can choose ['conv1', 'conv2']
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [23]:
target = 'mnist-0'
class_id = 0
bottleneck = 'conv2'
concept = 'imagenet-blue'

grad_lst = []
for i in range(20):
    acts, examples = act_generator.get_activations_for_noised_concept(target, bottleneck, noiselevel = i*0.01)
    act = np.expand_dims(acts[0],0)
    grad = mymodel.get_gradient(act, [class_id], bottleneck, examples[0])
    grad_lst.append(grad)

In [12]:
cav = get_cav(cav_dir, bottleneck, concept)

In [24]:
tcav_score = []
for grad in grad_lst:
    grad = np.reshape(grad,-1)
    tcav = [np.dot(grad,c)<0 for c in cav]
    tcav_score.append(np.array(tcav).sum()/len(tcav))